In [57]:
import json
import pandas as pd
import traceback
import requests
import re
from bs4 import BeautifulSoup
#from .dto import *

In [58]:
response = requests.get(
    url="https://play.google.com/store/apps/details",
    params={
        "id": "eco.app.libropia.v2",
        "hl": "ko",
    },
    headers={ "accept-encoding": "gzip, deflate, br" },
    cookies=None,
)

In [59]:
app_details = response.text
contents = app_details

# Find review data service keys

In [60]:
bs = BeautifulSoup(app_details, "html.parser")
scripts = bs.select("script")

In [61]:
services_regex = r"var AF_dataServiceRequests = ([^;]*)"

In [62]:
services_match = re.findall(services_regex, contents, re.MULTILINE)
services = services_match[0]

In [63]:
review_ds_key_regex = r"\'(ds:[0-9]+)\' : {id:\'oCPfdb\'"
review_ds_keys = re.findall(review_ds_key_regex, services, re.MULTILINE)

In [64]:
review_ds_keys

['ds:8']

# Parse review in a row

In [65]:
import datetime
def parse_review(review_row):
    writed_time = datetime.datetime.utcfromtimestamp(review_row[5][0])
    writed_time = writed_time.strftime("%Y-%m-%d %H:%M")

    return {
        "review_id": review_row[0],
        "author": review_row[1][0],
        "review_text": review_row[4],
        "rating": review_row[2],
        "writed_time": writed_time,
    }

# Scrap review in first page

In [66]:
review_data = []

In [67]:
import json
for script in scripts:
    ds_key_regex = r"AF_initDataCallback\({key: \'(ds:\d+)\',"
    ds_keys = re.findall(ds_key_regex, script.text, re.MULTILINE)
    if len(ds_keys) > 0 and ds_keys[0] in review_ds_keys:
        review_data_regex = r"data:([\[].*[\]]), sideChannel: {}}\);$"
        review_data_str = re.findall(review_data_regex, script.text, re.MULTILINE)[0]
        review_data += json.loads(review_data_str)
        break

In [68]:
list(map(parse_review, review_data[0]))

[{'review_id': 'c507ddcb-7c62-46b0-a393-bf6957fee82d',
  'author': 'Yohan Hong',
  'review_text': '앱 자체가 너무 무거워서 구동되는데 애로사항이 있고요. 무엇보다 접근권한이 뭐 이리 많이 필요하게 해놓았는지... 단순히 도서관에서 책빌리고 관리하는 용도인데 불구하고 필요 이상으로 (위치정보 부터 개인통화정보 등등) 너무 많은 종류의 개인정보를 가져가는 개발자분의 의도가 뭔지 매우 궁금합니다. 지금이라도 보다 라이트 하게 만들어서 오류를 줄여주시고, 필요이상의 개인정보를 요구하는 앱의 설정을 속히 변경하여 주시기 바랍니다.',
  'rating': 2,
  'writed_time': '2020-05-22 11:09'},
 {'review_id': 'bfc55f3d-ed31-4b7b-9b4e-d807857c7fab',
  'author': '박윤하',
  'review_text': '대출이력에서 읽었던 책을 확인하고 대여하는데 수학도둑 같은건 시리즈가 80번 이상대인데 숫자로 검색하면 나오지도 않고 너무 불편해요. 내림차순이나 오름차순같은 상세검색 기능이 있었으면으면 좋겠어요. 그리고 처음 검색할때 도서관 선택가능했으면 좋겠어요. 처음에 무조건 통합도서관 검색이라 불편해요.',
  'rating': 3,
  'writed_time': '2022-12-13 05:07'},
 {'review_id': '7d649b68-d65b-42c8-9634-9acfc223392a',
  'author': '유다은 (냥냥)',
  'review_text': '정말 편리하고 아주 소중히 여기는 앱이에요! 근데 검색을 할 때 앱이 계속 튕겨요..! 그 점이랑 도서를 검색할 때 필터 기능을 넣어서 유아 열람실과 어린이 열람실 같은 책들을 분류할 수 있었으면 좋겠어요~´ ˘ `',
  'rating': 5,
  'writed_time': '2023-01-25 11:34'},
 {'review_id': '1a3a

In [69]:
next_page = review_data[1][1]
next_page == "Cm0KawppMCwxMDAwMDAwLjM1NzU2ODQ0MjgsMTMwNDc4OTMxOTI3LCJodHRwOi8vbWFya2V0LmFuZHJvaWQuY29tL2RldGFpbHM_aWQ9djI6ZWNvLmFwcC5saWJyb3BpYS52MjoxIiwxLGZhbHNl"

True

# Scrap review in other pages

In [70]:
f_req_format = '[[["oCPfdb","[null,[2,1,[100,null,\\"%s\\"]],[\\"%s\\",7]]",null,"generic"]]]'

In [71]:
print(f_req_format % (next_page, "eco.app.libropia.v2"))

[[["oCPfdb","[null,[2,1,[100,null,\"Cm0KawppMCwxMDAwMDAwLjM1NzU2ODQ0MjgsMTMwNDc4OTMxOTI3LCJodHRwOi8vbWFya2V0LmFuZHJvaWQuY29tL2RldGFpbHM_aWQ9djI6ZWNvLmFwcC5saWJyb3BpYS52MjoxIiwxLGZhbHNl\"]],[\"eco.app.libropia.v2\",7]]",null,"generic"]]]


In [72]:
response = requests.post(
    url="https://play.google.com/_/PlayStoreUi/data/batchexecute",
    params={
        "rpcids": "oCPfdb",
        "hl": "ko",
    },
    data={
        "f.req": f_req_format % (next_page, "eco.app.libropia.v2")
    },
    headers={ "accept-encoding": "gzip, deflate, br" },
    cookies=None,
)

In [73]:
review_data = json.loads(json.loads(response.text[6:])[0][2])

In [77]:
review_data[0][0]

['35f3c25d-c8ed-4fb9-b02e-038dab51d8bb',
 ['aprilaroma yi',
  [None,
   2,
   None,
   [None,
    None,
    'https://play-lh.googleusercontent.com/a/AEdFTp4lOmfSYz8i_6hjQidxvck79jROqYzot5gl9lBg=mo']]],
 1,
 None,
 '업데이트 후, 계속 정상적인 구동이 되지 않네요~ 수정해주세요. 앱삭제, 다운 거듭해도 잠시 구동되고는 다음 사용 때엔 어김없이 삐그덕거리는지 벌써 일주일이 되어갑니다.',
 [1594020984, 697000000],
 5,
 None,
 None,
 ['116764572684505491886',
  'aprilaroma yi',
  None,
  [[None,
    2,
    None,
    [None,
     None,
     'https://play-lh.googleusercontent.com/a/AEdFTp4lOmfSYz8i_6hjQidxvck79jROqYzot5gl9lBg=mo']],
   True],
  [None,
   2,
   None,
   [None,
    None,
    'https://play-lh.googleusercontent.com/Yq7oyNIvAAkuc69fG51sbAQS4otJxbObbt3xdr8tXxXyUdq4tVGtfgeKuptveGdP1srxaHVrNPzOYcfaEQ']]],
 '3.2.44']

In [74]:
review_data[1]

[None,
 'CmwKagpoMCwxMDAwMDAwLjIyNzc1NjM5NTksODg1ODk4NDI0MTIsImh0dHA6Ly9tYXJrZXQuYW5kcm9pZC5jb20vZGV0YWlscz9pZD12MjplY28uYXBwLmxpYnJvcGlhLnYyOjEiLDEsZmFsc2U']

# Use default-scraper

In [85]:
from default_scraper.googleplay.review.parser import GooglePlayReviewParser

parser = GooglePlayReviewParser("eco.app.libropia.v2")
parser.run()

TypeError: 'NoneType' object is not subscriptable